<a href="https://www.kaggle.com/code/israaelmorsy33/ear-282-7-6-aug-split-true2?scriptVersionId=136599449" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#libraries
import tensorflow.keras 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense , Activation, Flatten , Conv2D , MaxPool2D ,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy   #for binary classification only
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau  #for stop over fitting
from tensorflow.keras import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report ,confusion_matrix,ConfusionMatrixDisplay
import numpy as np 
import shutil
import os
import math
import seaborn as sns
import pandas as pd


In [ ]:
main_direction_path = "/kaggle/input/ear-282image-7class-6class-aug-split/eardrum 282image 7class - 6class aug split"
print(main_direction_path)

In [ ]:
train_folder_path = main_direction_path + '/train/'
val_folder_path = main_direction_path + '/val/'
test_folder_path = main_direction_path + '/test/'

In [ ]:
#for train
aom_path_train_path = train_folder_path + '/AOM/'
csom_path_train_path = train_folder_path +'/CSOM/'
earwax_path_train_path = train_folder_path + '/Earwax/'
normal_path_train_path = train_folder_path +'/Normal/'
others_path_train_path = train_folder_path + '/others/'
otitisexterna_path_train_path = train_folder_path +'/OtitisEksterna/'

#for train
aom_path_val_path = val_folder_path + '/AOM/'
csom_path_val_path = val_folder_path +'/CSOM/'
earwax_path_val_path = val_folder_path + '/Earwax/'
normal_path_val_path = val_folder_path +'/Normal/'
others_path_val_path = val_folder_path + '/others/'
otitisexterna_path_val_path = val_folder_path +'/OtitisEksterna/'

#for test
aom_path_test_path  = test_folder_path + '/AOM/'
csom_path_test_path  = test_folder_path +'/CSOM/'
earwax_path_test_path  = test_folder_path + '/Earwax/'
normal_path_test_path  = test_folder_path +'/Normal/'
others_path_test_path  = test_folder_path + '/others/'
otitisexterna_path_test_path  = test_folder_path +'/OtitisEksterna/'


In [ ]:
totatl_train_images = (len(os.listdir(aom_path_train_path)) +
                       len(os.listdir(csom_path_train_path)) +
                       len(os.listdir(earwax_path_train_path)) +
                       len(os.listdir(normal_path_train_path)) +
                       len(os.listdir(others_path_train_path)) +
                       len(os.listdir(otitisexterna_path_train_path)))

print(totatl_train_images)

In [ ]:
totatl_val_images = (len(os.listdir(aom_path_val_path)) +
                       len(os.listdir(csom_path_val_path)) +
                       len(os.listdir(earwax_path_val_path)) +
                       len(os.listdir(normal_path_val_path)) +
                       len(os.listdir(others_path_val_path)) +
                       len(os.listdir(otitisexterna_path_val_path)))

print(totatl_val_images)

In [ ]:
totatl_test_images = (len(os.listdir(aom_path_test_path)) +
                       len(os.listdir(csom_path_test_path)) +
                       len(os.listdir(earwax_path_test_path)) +
                       len(os.listdir(normal_path_test_path)) +
                       len(os.listdir(others_path_test_path)) +
                       len(os.listdir(otitisexterna_path_test_path)))

print(totatl_test_images)

In [ ]:
classes_name = list(os.listdir(train_folder_path))
classes_name

In [ ]:
x = np.array([len(os.listdir(aom_path_train_path)),len(os.listdir(csom_path_train_path)),
              len(os.listdir(earwax_path_train_path)),len(os.listdir(normal_path_train_path)),
              len(os.listdir(others_path_train_path)),len(os.listdir(otitisexterna_path_train_path))])
labels = ['aom', 'csom', 'earwax', 'normal', 'others', 'otitisexterna']

plt.pie(x, labels = labels, autopct = '%.0f%%', radius= 3, textprops={'fontsize': 16})
plt.show()

In [ ]:
"""The function ImageDataGenerator augments your image by iterating through 
image as your CNN is getting ready to process that image"""

img_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   #validation_split = 0.20,
                                   #preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
                                   horizontal_flip = True)



training_set = img_datagen.flow_from_directory(train_folder_path,
                                                     target_size = (224, 224),
                                                     batch_size = 32,
                                                     class_mode='categorical',
                                                    # subset="training"
                                                     )

validation_set = img_datagen.flow_from_directory(val_folder_path,   
                                                       
                                                       target_size=(224, 224),
                                                       batch_size=32,
                                                       class_mode='categorical',
                                                     #  subset="validation"
                                                       )

test_set = img_datagen.flow_from_directory(test_folder_path,
                                          target_size = (224, 224),
                                          batch_size = 32,
                                          shuffle=False
                                      )

In [ ]:
training_set.class_indices  

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = tf.keras.applications.MobileNet(input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax")
x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Dense(512, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Print network structure
base_model.summary()

In [ ]:
model.summary()

In [ ]:
from tensorflow import keras
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

opt1=tf.keras.optimizers.Adam( learning_rate=0.02)

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:

checkpoint=  ModelCheckpoint(filepath = 'eff_model.h5',monitor='val_accuracy',save_best_only=True,save_weights_only=True)
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=10, min_delta=0.001)

In [ ]:
#callbacks used in the training
checkpoint =ModelCheckpoint("Incp_model",monitor='val_accuracy',verbose=1,save_best_only =True ,save_weights_only = False,mode='auto')
anne=ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=10,verbose=1, mode="auto", min_lr=0.0000001)
early = EarlyStopping(monitor='val_accuracy', min_delta=0,patience=18,verbose=1,mode='auto')

In [ ]:
## start Train/Test 
#no. of iteration
batch_size = 32
hist = model.fit(training_set, steps_per_epoch = training_set.samples//batch_size,validation_data = validation_set,
                 validation_steps = validation_set.samples//batch_size,
                 epochs = 100,
                 #callbacks=[checkpoint,early,anne]
                 )

In [ ]:
# Confusion Matrix  & Pres  & Recall   & F1-Score

target_names = ['AOM', 'CSOM', 'Earwax','Normal','Otitisexterna','Others']
label_names = [0,1,2,3,4,5]

Y_pred = model.predict(test_set)
y_pred_class = np.argmax(Y_pred ,  axis = 1)

cm = confusion_matrix(test_set.classes, y_pred_class, labels = label_names)


print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred_class))

print('classification_Report')
print(classification_report(test_set.classes, y_pred_class, target_names=target_names))

disp = ConfusionMatrixDisplay(confusion_matrix= cm, display_labels=target_names)
disp = disp.plot(cmap=plt.cm.Blues, values_format = 'g')
plt.savefig("cf_mobile769975_9992test.png")
plt.show()

In [ ]:
loss, acc = model.evaluate(test_set)

In [ ]:
model.save('mobile769975_9992test.h5')

In [ ]:
true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys())
class_labels

In [ ]:
report = classification_report(true_classes, y_pred_class, target_names=class_labels)
print(report) 

In [ ]:
report_dic = classification_report(test_set.classes, y_pred_class, target_names=class_labels, output_dict=True)
print(report_dic)    

In [ ]:
#geting predictions on test dataset
#y_pred = np.argmax(Y_pred, axis=1)
targetnames = ['AOM', 'CSOM', 'Earwax', 'Normal', 'OtitisEksterna', 'others']
#getting the true labels per image 
y_true = test_set.classes
#getting the predicted labels per image 
y_prob=Y_pred
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(y_true)

In [ ]:
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc


In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred_class, average='weighted')))
print("Recall: "+ str(recall_score(y_true, y_pred_class, average='weighted')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred_class)))
print("weighted Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='weighted')))

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred_class, average='macro')))
print("Recall: "+ str(recall_score(y_true, y_pred_class, average='macro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred_class)))
print("Macro Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='macro')))

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred_class, average='micro')))
print("Recall: "+ str(recall_score(y_true, y_pred_class, average='micro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred_class)))
tpr={}
fpr={}
roc_auc={}
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_prob.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
print("Micro Roc score: " + str(roc_auc["micro"]))

In [ ]:
fpr = {}
tpr = {}
roc_auc = {}
for i in range(6):
    r = roc_auc_score(y_test[:, i], y_prob[:, i])
    print("The ROC AUC score of "+targetnames[i]+" is: "+str(r))

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = {}
tpr = {}
roc_auc = dict()
for i in range(6):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_prob[:, i], drop_intermediate=False)
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
plt.figure(figsize=(20,14))

plt.plot(fpr[0], tpr[0],'v-',label='AOM: ROC curve of (area = %0.3f)' % roc_auc[0])
plt.plot(fpr[1], tpr[1],'c',label='CSOM: ROC curve of (area = %0.3f)' % roc_auc[1])
plt.plot(fpr[2], tpr[2],'b',label='Earwax: ROC curve of (area = %0.3f)' % roc_auc[2])
plt.plot(fpr[3], tpr[3],'g',label='Normal: ROC curve of (area = %0.3f)' % roc_auc[3])
plt.plot(fpr[4], tpr[4],'y',label='OtitisEksterna: ROC curve of (area = %0.3f)' % roc_auc[4])
plt.plot(fpr[5], tpr[5],'o-',label='others: ROC curve of (area = %0.3f)' % roc_auc[5])

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic of %s'%targetnames[i])
plt.legend(loc="lower right")
plt.savefig("ROCmobile769975_9992test.png")
plt.show()

In [ ]:
cf_matrix = confusion_matrix(true_classes, y_pred_class)

plt.figure(figsize=(6,6))
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', cbar=True) 

ax.set_title('Confusion Matrix\n\n');
ax.set_xlabel('\nPredicted Category')
ax.set_ylabel('Actual Category ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(class_labels)
ax.yaxis.set_ticklabels(class_labels)

## Display the visualization of the Confusion Matrix.
plt.savefig("cf2_mobile769975_9992test.png")
plt.show()

In [ ]:
Classification_report = pd.DataFrame.from_dict(report_dic)

In [ ]:
Classification_report

In [ ]:
Classification_report.to_csv('classification_report_mobile769975_9992test.csv', index = True)

In [ ]:
import seaborn as sns

df = pd.read_csv('/kaggle/working/classification_report_mobile769975_9992test.csv', delimiter=',',index_col=[0])
plt.figure(figsize=(20,5))
sns.heatmap(df,cbar=False, annot=True, fmt=".4g", cmap= 'Blues', linewidths = 3 )
plt.savefig("cr_mobile769975_9992test.png")